## Model Deployment

- There are several ways to deploy a model.  This is a same notebook walking you through the process of 

In [ ]:
import azureml.core
from azureml.core import Workspace
import pandas as pd

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

In [ ]:
# Create a Python environment for the experiment (from a .yml file)
# Import required packages
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice, AciWebservice
from azureml.exceptions import WebserviceException

conda_yml_file = './'+ experiment_folder+ '/environment.yml'
env = Environment.from_conda_specification("experiment_env", conda_yml_file)
env.register(workspace=ws)
print('')

In [ ]:
model_name = 'titanic-model'
environment_name = 'experiment_env'
service_name = 'servicename'

In [43]:
## Register the model
## Prepare entry script
## Prepare inference configration
## deploy model
## test results

In [ ]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'prep_for_deploy'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

In [ ]:
aci_config = AciWebservice.deploy_configuration(
            cpu_cores = 1, 
            memory_gb = 2, 
            tags = {'model': 'diabetes remote training'},
            auth_enabled=True,
            enable_app_insights=True,
            collect_model_data=True)

In [6]:
## Define Inference Config

In [ ]:
%%writefile ./$experiment_folder/environment.yml

name: experiment_env
dependencies:
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas=1.1.5
- pip
- pip:
  - azureml-defaults
  - pyarrow
  - azureml-monitoring
  - azureml-interpret
  - inference-schema
  - joblib
  - azure-ml-api-sdk
  - seaborn

In [ ]:
from IPython.display import Image

Image(filename="./image/Capture.png")

## Notes on preparing for API Call

- Manually grab url & api key, you can get them programmically & you will later in the notebook
- Be familiar with where in teh UI they are located.

## DEPLOY ME: Update the init script

In [48]:
%%writefile ./$experiment_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd

def init():
    global model
    # Replace filename if needed.
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'titanic_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)

def run(data):
    dict= json.loads(data)
    df = json_normalize(dict['data']) 
    df['loc']= df['cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
    df['hasFamily'] = (df['sibsp'] > 0) | (df['parch'] > 0)
    #note we remove the survived column
    cols_to_keep = ['pclass','sex','age','embarked','loc','hasFamily']
    df = df[cols_to_keep]
    
    print(df.isnull().sum())
    y_pred = model.predict(df)
    print(type(y_pred))
    
    #return json.dumps(y_pred)
    result = {"result": y_pred.tolist()}
    return result

Overwriting ./prep_for_deploy/score.py


In [50]:
model = Model(ws, model_name)
env = Environment.get(ws, environment_name)
inference_config = InferenceConfig(entry_script='./prep_for_deploy/score.py', environment=env)

try:
    service = Webservice(ws, name=service_name)
    if service:
        service.delete()
except WebserviceException as e:
         print()

service = Model.deploy(ws, service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-02-28 16:08:55+00:00 Creating Container Registry if not exists.
2022-02-28 16:08:55+00:00 Registering the environment.
2022-02-28 16:08:56+00:00 Building image..
2022-02-28 16:14:24+00:00 Generating deployment configuration.
2022-02-28 16:14:25+00:00 Submitting deployment to compute..
2022-02-28 16:14:29+00:00 Checking the status of deployment servicename..
2022-02-28 16:17:44+00:00 Checking the status of inference endpoint servicename.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [51]:
print(service.state)

Healthy


In [52]:
endpoint = service.scoring_uri
print(endpoint)

http://b6c30ea8-240c-4d91-90cf-e3b18f3cb736.eastus.azurecontainer.io/score


In [53]:
keys = service.get_keys()
selected_key = keys[0]
print(selected_key)

ETRP33uJdZaKjDaogKgjUEBXJIShst1O


In [57]:
import json
url = endpoint
api_key = selected_key # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
import requests

def MakePrediction(df):
    if 'survived' in df:
        print('removing surived from dataset')
        df.drop(['survived'], axis=1, inplace=True)
    endpoint_url = url
    body = df.to_json(orient='records') 
    body = '{"data": ' + body + '}'
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


df = pd.read_csv('./Data/Hold.csv')
results = MakePrediction(df)
print(type(results))

val = results['result']
print(val)

removing surived from dataset
<class 'dict'>
[0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 

In [60]:
df = pd.read_csv('./Data/Hold.csv')
df.columns

Index(['passenger_id', 'pclass', 'name', 'sex', 'age', 'sibsp', 'parch',
       'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest',
       'survived'],
      dtype='object')

In [75]:
y_test  =  df['survived'].tolist()
y_pred  = val
results = pd.DataFrame({'y_test': y_test, 'y_pred': val})
results['y_test'] = results['y_test'].astype(int)
results

,y_test,y_pred
0,0,0
1,1,1
2,0,0
3,1,1
4,0,0
...,...,...
388,0,0
389,1,1
390,0,1
391,0,0


In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(acc)
print(prec)
print(recall)

0.7837150127226463
0.7310344827586207
0.6973684210526315
